In [319]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import wordnet 
from nltk.corpus import wordnet, stopwords   #Import wordnet from the NLTK
from nltk.tokenize import word_tokenize
import numpy as np

In [320]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin-1')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [321]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "Text"]
data.columns = DATASET_COLUMNS
print(data.shape)
data.head(30)

(5043715, 6)


,target,ids,date,flag,user,Text
0,0,1.467811e+09,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1.467811e+09,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1.467811e+09,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1.467811e+09,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1.467811e+09,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
5,0,1.467812e+09,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
6,0,1.467812e+09,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
7,0,1.467812e+09,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
8,0,1.467812e+09,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?
9,0,1.467812e+09,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing


In [322]:
# total colm = 6, remove colm 2 to 5
data.drop(['ids','date','flag','user'],axis = 1,inplace = True)
data.head(10)

,target,Text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew
5,0,Need a hug
6,0,@LOLTrish hey long time no see! Yes.. Rains a...
7,0,@Tatiana_K nope they didn't have it
8,0,@twittera que me muera ?
9,0,spring break in plain city... it's snowing


In [323]:
positive_data = data[data.target==4].iloc[:10000,:]
print(positive_data.shape)
negative_data = data[data.target==0].iloc[:10000,:]
print(negative_data.shape)

(10000, 2)
(10000, 2)


In [324]:
data = pd.concat([positive_data,negative_data],axis = 0, ignore_index=True)
print(data.shape)
data.head(1001)

(20000, 2)


,target,Text
0,4,@ruann thats whats up....and yes twitter is a ...
1,4,@TokyoNeon enjoy the view
2,4,@coy0te Morning...
3,4,"@igorcabbab Hala, who did the catching and the..."
4,4,"Guten Morgen Twitterland, hello world."
...,...,...
996,4,@roychandra Dave and Andy rock
997,4,@Magpie_Guy you wicked bigger! I'm sure you sh...
998,4,"@JessicaSain Enjoy Dubai, it's an incredible p..."
999,4,aspro clear + lemsip = rad


In [325]:
# Remove @users
data['Clean_Text'] = data['Text'].str.replace("@", "") 
data.head(10)

,target,Text,Clean_Text
0,4,@ruann thats whats up....and yes twitter is a ...,ruann thats whats up....and yes twitter is a c...
1,4,@TokyoNeon enjoy the view,TokyoNeon enjoy the view
2,4,@coy0te Morning...,coy0te Morning...
3,4,"@igorcabbab Hala, who did the catching and the...","igorcabbab Hala, who did the catching and the ..."
4,4,"Guten Morgen Twitterland, hello world.","Guten Morgen Twitterland, hello world."
5,4,@nicolerichie It sucks being away from family ...,nicolerichie It sucks being away from family =...
6,4,@kristie__ im following you now fag im cold :|,kristie__ im following you now fag im cold :|
7,4,@ScreenOrigami oder knut,ScreenOrigami oder knut
8,4,"@TashLikesCheese ahh, thanks Tash!","TashLikesCheese ahh, thanks Tash!"
9,4,@thetalesend Next time I go to LA you have to ...,thetalesend Next time I go to LA you have to g...


In [326]:
# Remove http or https
data['Clean_Text'] = data['Clean_Text'].str.replace(r"http\S+", "") 
data.head(30)

,target,Text,Clean_Text
0,4,@ruann thats whats up....and yes twitter is a ...,ruann thats whats up....and yes twitter is a c...
1,4,@TokyoNeon enjoy the view,TokyoNeon enjoy the view
2,4,@coy0te Morning...,coy0te Morning...
3,4,"@igorcabbab Hala, who did the catching and the...","igorcabbab Hala, who did the catching and the ..."
4,4,"Guten Morgen Twitterland, hello world.","Guten Morgen Twitterland, hello world."
5,4,@nicolerichie It sucks being away from family ...,nicolerichie It sucks being away from family =...
6,4,@kristie__ im following you now fag im cold :|,kristie__ im following you now fag im cold :|
7,4,@ScreenOrigami oder knut,ScreenOrigami oder knut
8,4,"@TashLikesCheese ahh, thanks Tash!","TashLikesCheese ahh, thanks Tash!"
9,4,@thetalesend Next time I go to LA you have to ...,thetalesend Next time I go to LA you have to g...


In [327]:
#Removing Punctuations, Numbers, and Special Characters
data['Clean_Text'] = data['Clean_Text'].str.replace("[^a-zA-Z]", " ") 
data.head()

,target,Text,Clean_Text
0,4,@ruann thats whats up....and yes twitter is a ...,ruann thats whats up and yes twitter is a c...
1,4,@TokyoNeon enjoy the view,TokyoNeon enjoy the view
2,4,@coy0te Morning...,coy te Morning
3,4,"@igorcabbab Hala, who did the catching and the...",igorcabbab Hala who did the catching and the ...
4,4,"Guten Morgen Twitterland, hello world.",Guten Morgen Twitterland hello world


In [328]:
# transform all letters into lower case,
data['Clean_Text'] = data['Clean_Text'].str.lower()
data.head(30)

,target,Text,Clean_Text
0,4,@ruann thats whats up....and yes twitter is a ...,ruann thats whats up and yes twitter is a c...
1,4,@TokyoNeon enjoy the view,tokyoneon enjoy the view
2,4,@coy0te Morning...,coy te morning
3,4,"@igorcabbab Hala, who did the catching and the...",igorcabbab hala who did the catching and the ...
4,4,"Guten Morgen Twitterland, hello world.",guten morgen twitterland hello world
5,4,@nicolerichie It sucks being away from family ...,nicolerichie it sucks being away from family ...
6,4,@kristie__ im following you now fag im cold :|,kristie im following you now fag im cold
7,4,@ScreenOrigami oder knut,screenorigami oder knut
8,4,"@TashLikesCheese ahh, thanks Tash!",tashlikescheese ahh thanks tash
9,4,@thetalesend Next time I go to LA you have to ...,thetalesend next time i go to la you have to g...


In [329]:
# remove any extra spaces around the sentences with newline character
data['Clean_Text'] = data['Clean_Text'].str.strip()
data.head(30)

,target,Text,Clean_Text
0,4,@ruann thats whats up....and yes twitter is a ...,ruann thats whats up and yes twitter is a c...
1,4,@TokyoNeon enjoy the view,tokyoneon enjoy the view
2,4,@coy0te Morning...,coy te morning
3,4,"@igorcabbab Hala, who did the catching and the...",igorcabbab hala who did the catching and the ...
4,4,"Guten Morgen Twitterland, hello world.",guten morgen twitterland hello world
5,4,@nicolerichie It sucks being away from family ...,nicolerichie it sucks being away from family ...
6,4,@kristie__ im following you now fag im cold :|,kristie im following you now fag im cold
7,4,@ScreenOrigami oder knut,screenorigami oder knut
8,4,"@TashLikesCheese ahh, thanks Tash!",tashlikescheese ahh thanks tash
9,4,@thetalesend Next time I go to LA you have to ...,thetalesend next time i go to la you have to g...


In [330]:
# Replace a short form of words with full formation, e.g., won’t is replaced by will not, can’t by cannot and so on.

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", str(phrase))
    phrase = re.sub(r"can\'t", "can not", str(phrase))

    # general
    phrase = re.sub(r"n\'t", " not", str(phrase))
    phrase = re.sub(r"\'re", " are", str(phrase))
    phrase = re.sub(r"\'s", " is", str(phrase))
    phrase = re.sub(r"\'d", " would", str(phrase))
    phrase = re.sub(r"\'ll", " will", str(phrase))
    phrase = re.sub(r"\'t", " not", str(phrase))
    phrase = re.sub(r"\'ve", " have", str(phrase))
    phrase = re.sub(r"\'m", " am", str(phrase))
    return phrase

In [331]:
#data = decontracted(data['Clean_Text'].str)
data['Clean_Text'] = data['Clean_Text'].apply(lambda x: decontracted(x))
data.head(30)

,target,Text,Clean_Text
0,4,@ruann thats whats up....and yes twitter is a ...,ruann thats whats up and yes twitter is a c...
1,4,@TokyoNeon enjoy the view,tokyoneon enjoy the view
2,4,@coy0te Morning...,coy te morning
3,4,"@igorcabbab Hala, who did the catching and the...",igorcabbab hala who did the catching and the ...
4,4,"Guten Morgen Twitterland, hello world.",guten morgen twitterland hello world
5,4,@nicolerichie It sucks being away from family ...,nicolerichie it sucks being away from family ...
6,4,@kristie__ im following you now fag im cold :|,kristie im following you now fag im cold
7,4,@ScreenOrigami oder knut,screenorigami oder knut
8,4,"@TashLikesCheese ahh, thanks Tash!",tashlikescheese ahh thanks tash
9,4,@thetalesend Next time I go to LA you have to ...,thetalesend next time i go to la you have to g...


In [332]:
data['Clean_Text'][27]

'where is my trainee'

In [333]:
data['tokenized_sents'] = data.apply(lambda row: nltk.word_tokenize(row['Clean_Text']), axis=1)

In [334]:
data['tokenized_sents'][27]

['where', 'is', 'my', 'trainee']

In [335]:
stop_words = set(stopwords.words('english'))
data['tokenized_sents'] = data['tokenized_sents'].apply(lambda x: [item for item in x if item not in stop_words])

In [336]:
data['tokenized_sents'][27]

['trainee']

In [337]:
def get_synonyms(input_lemma):
    synonyms = []
    for syn in wordnet.synsets(input_lemma):
        for lemma in syn.lemmas(): 
            synonyms.append(lemma.name()) 
    return synonyms

In [338]:
data['synonyms'] = data['tokenized_sents'].apply(lambda x: [get_synonyms(y) for y in x])

In [339]:
data['synonyms'][27]

[['trainee']]

In [340]:
def get_antonyms(input_lemma):
    antonyms = []
    for syn in wordnet.synsets(input_lemma):
        for lemma in syn.lemmas():
            if lemma.antonyms():
                antonyms.append(lemma.antonyms()[0].name())
    return antonyms

In [341]:
data['antonyms'] = data['tokenized_sents'].apply(lambda x: [get_antonyms(y) for y in x])
data['antonyms'][0]

[[], [], [], ['no'], [], [], ['old', 'worn'], []]

In [342]:
def get_hypernyms(input_lemma):
    hypernyms = []
    for syn in wordnet.synsets(input_lemma):
        for hypernym in syn.hypernyms():
            hypernyms.append(hypernym.lemmas()[0].name())
    return hypernyms

In [343]:
data['hypernyms'] = data['tokenized_sents'].apply(lambda x: [get_hypernyms(y) for y in x])
data['hypernyms'][0]

[[],
 [],
 [],
 ['affirmative'],
 ['sound', 'peep'],
 [],
 [],
 ['natural_object',
  'class',
  'experience',
  'people',
  'part',
  'concern',
  'group',
  'homo']]

In [344]:
def get_hyponyms(input_lemma):
    hyponyms = []
    for syn in wordnet.synsets(input_lemma):
        for hyponym in syn.hyponyms():
            hyponyms.append(hyponym.lemmas()[0].name())
    return hyponyms

In [345]:
data['hyponyms'] = data['tokenized_sents'].apply(lambda x: [get_hyponyms(y) for y in x])

In [346]:
data['hyponyms'][0]

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 ['closed_universe',
  'natural_order',
  'nature',
  'academia',
  'Grub_Street',
  'real_world',
  'admass']]

In [347]:
# Combine synonyms, antonyms, hypernyms and hyponyms
data['final_list'] = data['synonyms']+data['antonyms'] +data['hypernyms'] +data['hyponyms']

In [348]:
print(data['final_list'][0])

[[], [], [], ['yes'], ['chirrup', 'twitter', 'chitter', 'twitter'], ['creepy', 'creepy', 'creepy-crawly'], ['new', 'fresh', 'new', 'novel', 'raw', 'new', 'new', 'unexampled', 'new', 'new', 'newfangled', 'new', 'New', 'Modern', 'New', 'new', 'young', 'new', 'newly', 'freshly', 'fresh', 'new'], ['universe', 'existence', 'creation', 'world', 'cosmos', 'macrocosm', 'world', 'domain', 'world', 'reality', 'Earth', 'earth', 'world', 'globe', 'populace', 'public', 'world', 'world', 'worldly_concern', 'earthly_concern', 'world', 'earth', 'world', 'human_race', 'humanity', 'humankind', 'human_beings', 'humans', 'mankind', 'man', 'global', 'planetary', 'world', 'worldwide', 'world-wide'], [], [], [], ['no'], [], [], ['old', 'worn'], [], [], [], [], ['affirmative'], ['sound', 'peep'], [], [], ['natural_object', 'class', 'experience', 'people', 'part', 'concern', 'group', 'homo'], [], [], [], [], [], [], [], ['closed_universe', 'natural_order', 'nature', 'academia', 'Grub_Street', 'real_world', 'ad

In [349]:
def get_unique(input_list):
    unique = []
    for a in input_list:
        for i in a:
            if i not in  unique:
                unique.append(i)
    return unique

In [350]:
data['unique'] = data['final_list'].apply(lambda x: get_unique(x))

In [351]:
data['unique'][0]

['yes',
 'chirrup',
 'twitter',
 'chitter',
 'creepy',
 'creepy-crawly',
 'new',
 'fresh',
 'novel',
 'raw',
 'unexampled',
 'newfangled',
 'New',
 'Modern',
 'young',
 'newly',
 'freshly',
 'universe',
 'existence',
 'creation',
 'world',
 'cosmos',
 'macrocosm',
 'domain',
 'reality',
 'Earth',
 'earth',
 'globe',
 'populace',
 'public',
 'worldly_concern',
 'earthly_concern',
 'human_race',
 'humanity',
 'humankind',
 'human_beings',
 'humans',
 'mankind',
 'man',
 'global',
 'planetary',
 'worldwide',
 'world-wide',
 'no',
 'old',
 'worn',
 'affirmative',
 'sound',
 'peep',
 'natural_object',
 'class',
 'experience',
 'people',
 'part',
 'concern',
 'group',
 'homo',
 'closed_universe',
 'natural_order',
 'nature',
 'academia',
 'Grub_Street',
 'real_world',
 'admass']

In [352]:
data[['unique','target']].head()

,unique,target
0,"[yes, chirrup, twitter, chitter, creepy, creep...",4
1,"[enjoy, bask, relish, savor, savour, love, del...",4
2,"[coy, demure, overmodest, tellurium, Te, atomi...",4
3,"[catching, detection, espial, spying, spotting...",4
4,"[morgen, hello, hullo, hi, howdy, how-do-you-d...",4


In [353]:
print(len(data['unique'][:]))

20000


In [354]:
# Create Skip Gram model
import gensim   
from gensim.models import Word2Vec


model = Word2Vec(min_count=1, window = 2)
model.build_vocab(data['unique'][:19900])  # prepare the model vocabulary
model.train(data['unique'][:19900], total_examples=model.corpus_count, epochs=model.iter)
model.save("word2vec.model")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


In [318]:
for i in range(len(data['unique'])):
    for j in data['unique'][i]:
        if j == 'Livy':
            print(i)

19910


In [317]:
data['unique'][19910]

['departure',
 'going',
 'going_away',
 'leaving',
 'passing',
 'loss',
 'exit',
 'expiration',
 'release',
 'sledding',
 'travel',
 'go',
 'move',
 'locomote',
 'proceed',
 'go_away',
 'depart',
 'become',
 'get',
 'run',
 'pass',
 'lead',
 'extend',
 'sound',
 'function',
 'work',
 'operate',
 'run_low',
 'run_short',
 'survive',
 'last',
 'live',
 'live_on',
 'endure',
 'hold_up',
 'hold_out',
 'die',
 'decease',
 'perish',
 'pass_away',
 'expire',
 'kick_the_bucket',
 "cash_in_one's_chips",
 'buy_the_farm',
 'conk',
 'give-up_the_ghost',
 'drop_dead',
 'pop_off',
 'choke',
 'croak',
 'snuff_it',
 'belong',
 'start',
 'get_going',
 'blend',
 'blend_in',
 'fit',
 'rifle',
 'plump',
 'fail',
 'go_bad',
 'give_way',
 'give_out',
 'conk_out',
 'break',
 'break_down',
 'Livy',
 'Titus_Livius',
 'party',
 'political_party',
 'company',
 'later',
 'ulterior',
 'posterior',
 'late',
 'belated',
 'tardy',
 'recent',
 'former',
 'previous',
 'subsequently',
 'afterwards',
 'afterward',
 'afte

In [156]:
data['target'][10000]
'''
word = data['unique'][0]
if word in model.wv.vocab:
    print('present in vocabulary')
else:
    print('not in vocabulary')
'''

0

In [355]:
model = Word2Vec.load("word2vec.model")
#vectorizing the test sentence
vector1 = model.wv[data['unique'][19901]]
print(len(vector1))

433


In [195]:
data['unique'][1]

for word in data['unique'][1]:
    if word in model.wv.vocab:
        print('present in vocabulary')
    else:
        print('not in vocabulary')

present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in vocabulary
present in

In [356]:
# Average vector of sentence
#print(v1)
avg1 = sum(vector1)/len(vector1)

In [357]:
from scipy.spatial.distance import cosine

dict_max = {}
for i in range(0, 19900):
    #print(data['unique'][i])
    if len(data['unique'][i]) > 0:
        vector2 = model.wv[data['unique'][i]]
        avg2 = sum(vector2)/len(vector2)
        cosine_similarity = 1 - cosine(avg1, avg2)
        if cosine_similarity > 0:
            dict_max[i] = cosine_similarity
print('Maximum Cosine Similarity is : ', max(dict_max.values()))

max_key = max(dict_max, key=dict_max.get)
print(max_key)

Maximum Cosine Similarity is :  0.97793048620224
17470


In [358]:
if data['target'][max_key] == 0:
    print('Depressed')
else:
    print('Not depressed')

Depressed


In [388]:
y_test = []

for i in range(19901, 19911):
    y_test.append(data['target'][i])
print(y_test)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [306]:
vector1 = []
test_avg = []
for j in range(19901, 19903):
    for word in data['unique'][j]:
        if word in model.wv.vocab:
            vector1.append(model.wv[data['unique'][j]])

In [307]:
print(len(vector1))

741


In [308]:
test_avg = []

for vect in vector1:
    avg1 = sum(vect)/len(vect)
    test_avg.append(avg1)

In [309]:
print(len(test_avg[2]))

100


In [360]:
maximum = []
y_max = []

dict_max = {}
for j in range(19901, 19905):
    for word in data['unique'][j]:
        vector1 = model.wv[data['unique'][j]]
        avg1 = sum(vector1)/len(vector1)
        for i in range(0, 10):
            #print(data['unique'][i])
            if len(data['unique'][i]) > 0:
                vector2 = model.wv[data['unique'][i]]
                avg2 = sum(vector2)/len(vector2)
                cosine_similarity = 1 - cosine(avg1, avg2)
                if cosine_similarity > 0:
                    dict_max[i] = cosine_similarity
    y_max.append(max(dict_max, key=dict_max.get))

In [374]:
y_max = []

In [397]:
vector1 = model.wv[data['unique'][19910]]
avg1 = sum(vector1)/len(vector1)

dict_max = {}
for i in range(0, 19900):
    #print(data['unique'][i])
    if len(data['unique'][i]) > 0:
        vector2 = model.wv[data['unique'][i]]
        avg2 = sum(vector2)/len(vector2)
        cosine_similarity = 1 - cosine(avg1, avg2)
        if cosine_similarity > 0:
            dict_max[i] = cosine_similarity
print('Maximum Cosine Similarity is : ', max(dict_max.values()))

max_key = max(dict_max, key=dict_max.get)
y_max.append(max_key)

Maximum Cosine Similarity is :  0.994276762008667


In [398]:
print(y_max)

[17470, 625, 12979, 15840, 4850, 7324, 2228, 5043, 12599, 12040]


In [399]:
y_pred = []

for i in y_max:
    y_pred.append(data['target'][i])
#print(y_pred)

In [400]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score


precision, recall, fscore, support = score(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print('Acc: {:.4f}'.format(acc))
print('fscore: {}'.format(fscore))

Acc: 0.5000
fscore: [0.66666667 0.        ]


In [162]:
acc = accuracy_score(y_test, y_pred)
print('Acc: {:.4f}'.format(acc))

Acc: 0.7500


In [50]:
from sklearn.metrics import precision_recall_fscore_support as score

predicted = [1,2,3,4,5,1,2,1,1,4,5] 
y_test = [1,2,3,4,5,1,2,1,1,4,1]

precision, recall, fscore, support = score(y_test, predicted)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: [1.  1.  1.  1.  0.5]
recall: [0.8 1.  1.  1.  1. ]
fscore: [0.88888889 1.         1.         1.         0.66666667]
support: [5 2 1 2 1]
